In [17]:
setwd("/Users/micaelawiseman/PhD/Thesis/Winterlight")
source("/Users/micaelawiseman/PhD/Thesis/Winterlight/Code/WL_helperfuncs.r", encoding = "UTF-8")

Load winterlight data

In [18]:
# Read and preprocess speech datasets
library(readxl)
library(dplyr)
library(readr)
library(tidyr)

# Read datasets
# Read and preprocess speech data
WL <- speech_read_and_preprocess("~/PhD/Thesis/Data/WINTERLIGHT_Sunnybrook_rTMS_2023_11_03.csv")
WL_2 <- speech_read_and_preprocess("~/PhD/Thesis/Data/WINTERLIGHT_Sunnybrook_rTMSremote_2023_11_03.csv")

# Apply filters
WL <- WL[grep("^(TMS|MDD|MFB|OCD)", WL$participant_external_id), ]
WL_2 <- WL_2[!(WL_2$participant_external_id == "TMS039" & WL_2$session_label %in% c("V2", "V3", "V4")), ]
WL_2 <- WL_2[!(WL_2$participant_external_id == "TMS039b" & WL_2$session_label == "V1"), ]
WL_2$participant_external_id[WL_2$session_label %in% c("V2", "V3") & WL_2$participant_external_id == "TMS039b"] <- "TMS039"

# Combine datasets
WL_combined <- rbind(WL, WL_2)

# Processing participant_group
WL_combined$participant_group <- factor(
  ifelse(
    grepl("(^CTC|C_CTC)", WL_combined$participant_external_id), "Control",
    ifelse(grepl("^(TMS|MDD|MFB)", WL_combined$participant_external_id), "MDD",
    ifelse(grepl("^(OCD)", WL_combined$participant_external_id), "OCD",
    "Other" 
    )
  )
))

# Exclude specific participants
WL_combined <- WL_combined[WL_combined$participant_external_id != "CTC036" &
    WL_combined$participant_external_id != "CTC004" &
    WL_combined$participant_external_id != "CTC006" &
    WL_combined$participant_external_id != "CTC017" &
    WL_combined$participant_external_id != "CTC030" &
    WL_combined$participant_external_id != "CTC039" &
    WL_combined$participant_external_id != "CTC043" &
    WL_combined$participant_external_id != "CTC045" &
    WL_combined$participant_external_id != "CTC053" &
    WL_combined$participant_external_id != "CTC052" &
    WL_combined$participant_external_id != "CTC058" &
    WL_combined$participant_external_id != "CTC058_new" &
    WL_combined$participant_external_id != "CTC023" &
    WL_combined$participant_external_id != "CTB001", ] ## Remove participants with high QIDS

# Remove participants with baseline data post TMS 
WL_combined <- WL_combined[!(WL_combined$participant_external_id == "TMS049" & WL_combined$session_label == "V1"), ]

# Define remote participants and assign testing location
remote_participants <- c(
    "CTC001", "CTC015", "CTC021", "CTC028", "CTC013", "CTC030",
    "CTC034", "CTC036", "CTC045", "TMS038")

# Function to identify CTC046 and higher
is_ctc046_or_higher <- function(id) {
  if (grepl("^CTC", id)) {
    # Extract the numeric part of the ID and check if it is 46 or higher
    num_part <- as.numeric(sub("^CTC", "", id))
    return(num_part >= 46)
  }
  FALSE
}

# Apply the function to all participants and get those that are CTC046 or higher
ctc046_or_higher <- sapply(WL_combined$participant_external_id, is_ctc046_or_higher)
additional_ctc <- WL_combined$participant_external_id[ctc046_or_higher]


# Function to identify TMS040 and higher
is_tms040_or_higher <- function(id) {
    if (grepl("^TMS", id)) {
        # Extract the numeric part of the ID and check if it is 46 or higher
        num_part <- as.numeric(sub("^TMS", "", id))
        return(num_part >= 40)
    }
    FALSE
}

# Apply the function to all participants and get those that are CTC046 or higher
tms040_or_higher <- sapply(WL_combined$participant_external_id, is_tms040_or_higher)
additional_tms <- WL_combined$participant_external_id[tms040_or_higher]

remote_participants <- c(remote_participants, additional_ctc, additional_tms)

WL_combined$testing_location <- ifelse(WL_combined$participant_external_id %in% remote_participants, 
                                       "remote", "in-person")
WL_combined$testing_location <- ifelse(WL_combined$participant_external_id %in% c("TMS052", "TMS053"), 
                                       "remote", WL_combined$testing_location)

                                 

# Replace values in session_label column
WL_combined$session_label <- gsub("Baseline", "V1", WL_combined$session_label)

str(WL_combined)

'data.frame':	1081 obs. of  810 variables:
 $ X                                                          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ sample_id                                                  : int  51341 51342 51343 51344 51345 51346 51347 51348 51349 52366 ...
 $ participant_external_id                                    : chr  "OCD108" "OCD108" "OCD108" "OCD108" ...
 $ task_name                                                  : chr  "paragraph_reading" "picture_description" "picture_description" "phonemic_fluency" ...
 $ session_label                                              : chr  "V1" "V1" "V1" "V1" ...
 $ sample_datetime_completed_utc                              : chr  "2021-12-16 17:41:38" "2021-12-16 17:42:55" "2021-12-16 17:44:07" "2021-12-16 17:45:58" ...
 $ stimulus_filename                                          : chr  "paragraph_03" "01_WinterLight_Family_in_the_kitchen_web.png" "02_WinterLight_Living_room_web.png" "en_v2_instructions_phonemic_fluency_f.m4a"

Journalling only, baseline

In [19]:
WL_jou <-subset_by_task(WL_combined, "journaling")
WL_jou_bl <- subset_by_visit(WL_jou, "V1")
str(WL_jou_bl)

'data.frame':	154 obs. of  810 variables:
 $ X                                                          : int  7 8 16 17 43 44 70 71 97 98 ...
 $ sample_id                                                  : int  51347 51348 52372 52373 52399 52400 52426 52427 53088 53089 ...
 $ participant_external_id                                    : chr  "OCD108" "OCD108" "TMS001" "TMS001" ...
 $ task_name                                                  : chr  "journaling" "journaling" "journaling" "journaling" ...
 $ session_label                                              : chr  "V1" "V1" "V1" "V1" ...
 $ sample_datetime_completed_utc                              : chr  "2021-12-16 17:49:14" "2021-12-16 17:49:42" "2022-01-04 16:09:55" "2022-01-04 16:10:17" ...
 $ stimulus_filename                                          : chr  "en_instruction_journal_feeling.mp3" "en_instruction_yesterday.mp3" "en_instruction_journal_feeling.mp3" "en_instruction_yesterday.mp3" ...
 $ terminal_state          

Remove columns with mostly or all NAs, 0 variance, and speech features irrelevant for connected speech (jitter, shimmer, hnr), or remote recording (mfccs subject to background noise)

In [27]:
WL_jou_clean <- clean_dataframe(WL_jou_bl, na_threshold = 0.6)
cols_to_remove <- grep("^(jitter|shimmer|HNR)", names(data), value = TRUE)
cols_to_remove <- c(cols_to_remove, "X", "sample_id", "task_name", "session_label", "sample_datetime_completed_utc", "terminal_state")
WL_jou_clean <- WL_jou_clean[ , !(names(WL_jou_clean) %in% cols_to_remove)]
write.csv(WL_jou_clean, "~/PhD/Thesis/Data/WL_jou_clean.csv", row.names = FALSE)
head(WL_jou_clean)

Columns with >= 60 % NAs: terminal_message cdr_name_address_correct_words_total cdr_name_address_correct_words_unique cdr_name_address_incorrect_words_total filled_pause_duration global_coherence_GloVe_100_avg_dist global_coherence_GloVe_100_max_dist global_coherence_GloVe_100_min_dist global_coherence_GloVe_200_avg_dist global_coherence_GloVe_200_max_dist global_coherence_GloVe_200_min_dist global_coherence_GloVe_300_avg_dist global_coherence_GloVe_300_max_dist global_coherence_GloVe_300_min_dist global_coherence_GloVe_50_avg_dist global_coherence_GloVe_50_max_dist global_coherence_GloVe_50_min_dist global_coherence_Google_300_avg_dist global_coherence_Google_300_max_dist global_coherence_Google_300_min_dist global_coherence_fastText_300_avg_dist global_coherence_fastText_300_max_dist global_coherence_fastText_300_min_dist info_units_bool_count info_units_bool_count_action info_units_bool_count_location info_units_bool_count_object info_units_bool_count_subject info_units_count info_u

,participant_external_id,stimulus_filename,tag_audio_quality,tag_background_noise,tag_participant_accent,tag_participant_clarity,tag_clinician_interference,tag_unsolicited_phi,ADJP_.._JJ,Lu_C,...,total_duration_audio,total_duration_speech,total_words,uh,unfilled_pauses,zcr_kurtosis,zcr_mean,zcr_skewness,participant_group,testing_location
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
7,OCD108,en_instruction_journal_feeling.mp3,0,1,0,0,0,0,0.031250000,1.285714,...,20.07,0.53810303,0.9000000,0.01666667,0.10000000,16.691086,0.04620508,4.323319,OCD,in-person
8,OCD108,en_instruction_yesterday.mp3,0,1,1,0,0,0,0.000000000,1.000000,...,20.07,0.26406896,0.8064516,0.00000000,0.19354839,20.049571,0.03998734,4.695697,OCD,in-person
16,TMS001,en_instruction_journal_feeling.mp3,0,1,1,1,1,0,0.037037037,1.500000,...,26.54,0.35793659,0.7241379,0.13793103,0.20689655,13.078970,0.05549323,3.883165,MDD,in-person
17,TMS001,en_instruction_yesterday.mp3,1,1,1,1,1,0,0.000000000,1.500000,...,10.17,0.08849558,0.7727273,0.09090909,0.13636364,16.251423,0.04716553,4.272168,MDD,in-person
43,TMS002,en_instruction_journal_feeling.mp3,0,0,0,1,0,0,0.006622517,2.333333,...,38.29,0.94801755,1.0208333,0.01388889,0.04861111,9.879678,0.06753448,3.446691,MDD,in-person
44,TMS002,en_instruction_yesterday.mp3,1,1,0,0,0,0,0.000000000,1.100000,...,27.70,0.84294310,0.9354839,0.04301075,0.04301075,9.409625,0.06976112,3.377814,MDD,in-person


In [ ]:
# Identify missing values per column
missing_values_summary <- sapply(WL_jou_clean, function(x) sum(is.na(x)))
cat("Missing values per column:\n")
print(missing_values_summary)

# Verify that the summary shows correct information
if(all(missing_values_summary == 0)) {
  cat("No missing values found in any column.\n")
} else {
  cat("Missing values found in the dataset.\n")
}

# Find rows with any missing value
rows_with_na <- apply(WL_jou_clean, 1, function(x) any(is.na(x)))

# Extract data for rows with missing values
data_with_na <- WL_jou_clean[rows_with_na, ]

# Count the number of NAs in each row of the subset
data_with_na$na_count <- rowSums(is.na(data_with_na))

# Print rows (participants) with missing values and their count of NAs
cat("Participants with missing values and their NA counts:\n")
print(data_with_na[, c("participant_external_id", "na_count")])


KNN imputation 

In [28]:
# Replace positive infinity values with NA
WL_jou_clean[WL_jou_clean == Inf] <- NA

# Replace negative infinity values with NA
WL_jou_clean[WL_jou_clean == -Inf] <- NA

head(WL_jou_clean)

,participant_external_id,stimulus_filename,tag_audio_quality,tag_background_noise,tag_participant_accent,tag_participant_clarity,tag_clinician_interference,tag_unsolicited_phi,ADJP_.._JJ,Lu_C,...,total_duration_audio,total_duration_speech,total_words,uh,unfilled_pauses,zcr_kurtosis,zcr_mean,zcr_skewness,participant_group,testing_location
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
7,OCD108,en_instruction_journal_feeling.mp3,0,1,0,0,0,0,0.031250000,1.285714,...,20.07,0.53810303,0.9000000,0.01666667,0.10000000,16.691086,0.04620508,4.323319,OCD,in-person
8,OCD108,en_instruction_yesterday.mp3,0,1,1,0,0,0,0.000000000,1.000000,...,20.07,0.26406896,0.8064516,0.00000000,0.19354839,20.049571,0.03998734,4.695697,OCD,in-person
16,TMS001,en_instruction_journal_feeling.mp3,0,1,1,1,1,0,0.037037037,1.500000,...,26.54,0.35793659,0.7241379,0.13793103,0.20689655,13.078970,0.05549323,3.883165,MDD,in-person
17,TMS001,en_instruction_yesterday.mp3,1,1,1,1,1,0,0.000000000,1.500000,...,10.17,0.08849558,0.7727273,0.09090909,0.13636364,16.251423,0.04716553,4.272168,MDD,in-person
43,TMS002,en_instruction_journal_feeling.mp3,0,0,0,1,0,0,0.006622517,2.333333,...,38.29,0.94801755,1.0208333,0.01388889,0.04861111,9.879678,0.06753448,3.446691,MDD,in-person
44,TMS002,en_instruction_yesterday.mp3,1,1,0,0,0,0,0.000000000,1.100000,...,27.70,0.84294310,0.9354839,0.04301075,0.04301075,9.409625,0.06976112,3.377814,MDD,in-person


In [40]:
numeric_data <- WL_jou_clean[sapply(WL_jou_clean, is.numeric)]

# Calculate the correlation matrix using only numeric data
cor_matrix <- cor(numeric_data, use = "complete.obs")
# Find highly correlated pairs (e.g., correlation > 0.9)
highly_cor <- findCorrelation(cor_matrix, cutoff = 0.9)

str(WL_jou_clean_reduced)

'data.frame':	154 obs. of  180 variables:
 $ tag_audio_quality                       : int  0 0 0 1 0 1 0 0 0 0 ...
 $ tag_background_noise                    : int  1 1 1 1 0 1 1 1 1 1 ...
 $ tag_participant_accent                  : int  0 1 1 1 0 0 0 0 0 0 ...
 $ tag_participant_clarity                 : int  0 0 1 1 1 0 1 1 0 0 ...
 $ tag_clinician_interference              : int  0 0 1 1 0 0 0 0 0 1 ...
 $ tag_unsolicited_phi                     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ ADJP_.._JJ                              : num  0.03125 0 0.03704 0 0.00662 ...
 $ Lu_CN                                   : num  0.286 1 0.75 1 2.111 ...
 $ Lu_CN.C                                 : num  0.222 1 0.5 0.667 0.905 ...
 $ Lu_CN.T                                 : num  0.333 1.5 1 2 4.75 ...
 $ Lu_CP                                   : num  0 0 0.25 0.5 0.111 ...
 $ Lu_CP.C                                 : num  0 0 0.1667 0.3333 0.0476 ...
 $ Lu_CT                                   : num  0.28

In [46]:
# 1. Identify variables with missing values
variables_with_missing <- names(WL_jou_clean)[colSums(is.na(WL_jou_clean)) > 0]
print(variables_with_missing)
# Adjust it to retain variables with missing values
reduced_columns_to_remove <- setdiff(highly_cor, variables_with_missing)
# 2. Create the reduced dataset
reduced_data <- WL_jou_clean[, !(names(WL_jou_clean) %in% reduced_columns_to_remove)]

str(reduced_data)

# Perform imputation on the reduced dataset
# 3. MICE imputation (example, adjust parameters as needed)
imputed_data <- mice(reduced_data, m=5, method='pmm', maxit=50)

# Complete the dataset with imputed values
completed_data <- complete(imputed_data)

# 4. Merge imputed values back into the original dataset
for(variable in variables_with_missing) {
  WL_jou_clean[is.na(WL_jou_clean[, variable]), variable] <- completed_data[, variable]
}

####CONTACT MICHALE SPILKA 

 [1] "MATTR_10"                               
 [2] "MATTR_20"                               
 [3] "MATTR_30"                               
 [4] "MATTR_40"                               
 [5] "MATTR_50"                               
 [6] "NOUN_age_of_acquisition"                
 [7] "NOUN_familiarity"                       
 [8] "NOUN_frequency"                         
 [9] "NOUN_imageability"                      
[10] "NOUN_sentiment_arousal"                 
[11] "NOUN_sentiment_dominance"               
[12] "NOUN_sentiment_valence"                 
[13] "VERB_age_of_acquisition"                
[14] "VERB_familiarity"                       
[15] "VERB_frequency"                         
[16] "VERB_imageability"                      
[17] "VERB_sentiment_arousal"                 
[18] "VERB_sentiment_dominance"               
[19] "VERB_sentiment_valence"                 
[20] "age_of_acquisition"                     
[21] "ave_cos_dist"                           
[22] "familia

'data.frame':	154 obs. of  384 variables:
 $ participant_external_id                 : chr  "OCD108" "OCD108" "TMS001" "TMS001" ...
 $ stimulus_filename                       : chr  "en_instruction_journal_feeling.mp3" "en_instruction_yesterday.mp3" "en_instruction_journal_feeling.mp3" "en_instruction_yesterday.mp3" ...
 $ tag_audio_quality                       : int  0 0 0 1 0 1 0 0 0 0 ...
 $ tag_background_noise                    : int  1 1 1 1 0 1 1 1 1 1 ...
 $ tag_participant_accent                  : int  0 1 1 1 0 0 0 0 0 0 ...
 $ tag_participant_clarity                 : int  0 0 1 1 1 0 1 1 0 0 ...
 $ tag_clinician_interference              : int  0 0 1 1 0 0 0 0 0 1 ...
 $ tag_unsolicited_phi                     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ ADJP_.._JJ                              : num  0.03125 0 0.03704 0 0.00662 ...
 $ Lu_C                                    : num  1.29 1 1.5 1.5 2.33 ...
 $ Lu_C.S                                  : num  1.29 1 1.5 1.5 2.33 ...
 $ L

ERROR: Error in solve.default(xtx + diag(pen)): system is computationally singular: reciprocal condition number = 5.08666e-17


In [41]:
# Install and load the necessary package
install.packages("mice")
library(mice)

# Perform MICE imputation
# Method "norm.predict" uses Bayesian linear regression for imputation
# You can adjust the m argument for the number of multiple imputations
imputed_data <- mice(WL_jou_clean_reduced, method = 'norm.predict', m = 5, maxit = 10)

# Complete the data by choosing one of the imputed datasets
completed_data <- complete(imputed_data, 1)

# Alternatively, you can explore and use the imputed datasets for analysis



The downloaded binary packages are in
	/var/folders/0w/9q69h2j54flcw9syyjdzrb3m0000gn/T//RtmpwHtru4/downloaded_packages

 iter imp variable
  1   1  MATTR_10*  MATTR_50*  NOUN_age_of_acquisition*  NOUN_familiarity  NOUN_frequency  NOUN_imageability  NOUN_sentiment_arousal  NOUN_sentiment_dominance  NOUN_sentiment_valence  VERB_age_of_acquisition*  VERB_familiarity  VERB_frequency  VERB_imageability  VERB_sentiment_arousal  VERB_sentiment_dominance  VERB_sentiment_valence  age_of_acquisition  ave_cos_dist  familiarity  graph_avg_total_degree  graph_diameter_undirected  graph_lsc  graph_num_edges  graph_pe_undirected  honore  imageability  local_coherence_GloVe_100_max_dist  local_coherence_GloVe_200_min_dist  local_coherence_GloVe_50_avg_dist  local_coherence_Google_300_avg_dist  local_coherence_Google_300_max_dist  local_coherence_Google_300_min_dist  local_coherence_fastText_300_avg_dist  local_coherence_fastText_300_max_dist  local_coherence_fastText_300_min_dist  nv_ratio  prp_rati

Warning message:
"Number of logged events: 360"


Random forest with all visits, all task files, no feature selection 

In [35]:
library(caret)
library(randomForest)

# Set a seed for reproducibility
set.seed(123)

# Ensure the target variable is a factor
WL_jou_clean_imputed$participant_group <- as.factor(WL_jou_clean_imputed$participant_group)

# Center and scale 
preProcValues <- preProcess(WL_jou_clean_imputed, method = c("center", "scale"))

# Split the dataset into training and testing sets
trainIndex <- createDataPartition(WL_jou_clean_imputed$participant_group, p = 0.7, list = FALSE)
trainData <- WL_jou_clean_imputed[trainIndex, ]
testData <- WL_jou_clean_imputed[-trainIndex, ]

# Train the model using caret with Random Forest
fitControl <- trainControl(method = "repeatedcv", number = 10, repeats = 5) #repeated cross-validation for resampling 
rfFit <- train(participant_group ~ ., data = trainData, method = "rf", trControl = fitControl, preProcess = preProcValues, ntree = 500) # 500 random trees

# Print the model summary
print(rfFit)

# Predict on the testing set
predictions <- predict(rfFit, newdata = testData)

# Evaluate the model performance
confusionMatrix(predictions, testData$Group)


Loading required package: ggplot2

Loading required package: lattice

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


The following object is masked from 'package:dplyr':

    combine


Warning message in createDataPartition(WL_jou_clean_imputed$participant_group, p = 0.7, :
"Some classes have no records ( Other ) and these will be ignored"


ERROR: Error: pre-processing methods are limited to: BoxCox, YeoJohnson, expoTrans, invHyperbolicSine, center, scale, range, knnImpute, bagImpute, medianImpute, pca, ica, spatialSign, ignore, keep, remove, zv, nzv, conditionalX, corr
